# Exercise 4: Smoke Reconstruction

The goal of this exercise is to reconstruct 2D smoke simulations from only the projected smoke density.

The observations consist of the projected smoke densities along $x$ and $y$ over 20 time steps and the reconstruction must contain the 2D smoke reconstructions as well as the velocity fields.

In [ ]:
!pip install phiflow
from phi.flow import *

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.is_available())
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Insert your reconstruction algorithm / network here, replacing the next cell.

In [ ]:
''' DATASET GENERATION FOR EXERCISE 4'''
from phi.torch.flow import * 
from phi.field._grid import unstack_staggered_tensor
import os
import numpy as np
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# pads centered grid tensor to staggered size
def cen_to_stag(centered_grid):
    padded_values = math.pad(centered_grid.values, {'x':(0,1), 'y':(0,1)}, mode=math.extrapolation.ZERO)
    return padded_values

# reduces staggered size tensor to centered tensor
def reduce_to_cen(reduce_tensor):
    return math.pad(reduce_tensor, {'x':(0,-1), 'y':(0,-1)}, mode=math.extrapolation.ZERO)

In [ ]:
k=32 # number of discrete cells per dimsension
Nb = 100
Nt = 20
# generate random buoyant smoke spheres 
def generate_inflow(shape: math.Shape, bounds=Box(x=100, y=100)):
    centers = math.random_uniform(shape, channel(vector='x,y')) * bounds.size * (0.5, .3) + (25,10.)
    sizes =  math.random_uniform(shape) *4 + 3
    strengths = 1
    return CenteredGrid(SoftGeometryMask(Sphere(centers, radius=sizes)), 0, x=k, y=k, bounds=bounds) * strengths

# dataset size in batch dimension
INFLOW = generate_inflow(batch(batch=Nb) & instance(inflows=2)) 
velocity = StaggeredGrid((0, 0), 0, x=k, y=k, bounds=Box(x=100, y=100))  # or CenteredGrid(...)
smoke = CenteredGrid(INFLOW, extrapolation.BOUNDARY, x=k, y=k, bounds=Box(x=100, y=100))
pressure = None

# @math.jit_compile 
def step(v, s, p, dt=5.):
    s = advect.mac_cormack(s, v, dt) #+ INFLOW
    buoyancy = s * (0, 0.1) @ v  # resamples smoke to velocity sample points
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, (), Solve('auto', 1e-5, 0))
    return v, s, p

smokes, velocities, pxs, pys = [], [], [], []
for i in range(Nt):
    velocity, smoke, pressure = step(velocity, smoke, pressure)
    smokes.append(cen_to_stag(smoke).native(['batch','x','y']))
    velocities.append(velocity.staggered_tensor().native(['batch', 'x', 'y','vector']))
    px =torch.mean(smokes[-1], axis=1)
    py =torch.mean(smokes[-1], axis=2)
    pxs.append(px)
    pys.append(py)

# input data is stored with following dimensions: [batch, time, axis, (projection_x, projection_y)]
# ouput data is stored with following dimensions: [batch, time, x, y, (smoke, vx, vy)]
smokes = torch.permute(torch.stack(smokes),[1,0,2,3])
velocities = torch.permute(torch.stack(velocities),[1,0,2,3,4])
pxs =  torch.permute(torch.stack(pxs),[1,0,2])
pys =  torch.permute(torch.stack(pys),[1,0,2])
input_data = torch.stack([pxs,pys], dim=-1).cpu().numpy()
output_data = torch.cat([torch.unsqueeze(smokes[:,:], -1), velocities[:,:]],-1).cpu().numpy()

np.save('training_input_ex4',input_data)
np.save('training_output_ex4',output_data)
np.savetxt('training_input_ex4.csv', input_data.flatten(), delimiter=',')
np.savetxt('training_output_ex4.csv', output_data.flatten(), delimiter=',')

In [ ]:
''' SIMULATOR STEP BASED ON TENSORS '''
from phi.field._grid import unstack_staggered_tensor
state =math.tensor(output_data[:10], batch('batch', 'time'), spatial('x,y'), channel(features='rho,vx,vy'))
k=32
def cen_to_stag(centered_grid):
    padded_values = math.pad(centered_grid.values, {'x':(0,1), 'y':(0,1)}, mode=math.extrapolation.ZERO)
    return padded_values

def reduce_to_cen(reduce_tensor):
    return math.pad(reduce_tensor, {'x':(0,-1), 'y':(0,-1)}, mode=math.extrapolation.BOUNDARY)

def step(state, dt=5.):
    s = CenteredGrid(reduce_to_cen(state.features['rho']), extrapolation.BOUNDARY, x=k, y=k, bounds=Box(x=100, y=100))
    unstacked_velocity = unstack_staggered_tensor(math.rename_dims(state.features['vx,vy'], 'features', 'vector'), extrapolation=math.extrapolation.ZERO)
    v = StaggeredGrid(unstacked_velocity, 0, x=k, y=k, bounds=Box(x=100, y=100))
    s = advect.mac_cormack(s, v, dt) #+ INFLOW
    buoyancy = s * (0, 0.1) @ v  # resamples smoke to velocity sample points
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v,_= fluid.make_incompressible(v, (), Solve('auto', 1e-5, 0))
    v = v.staggered_tensor()
    return math.stack([cen_to_stag(s), v.vector['x'], v.vector['y']], channel(features='rho,vx,vy')) 

In [ ]:
# CHECK IF RESULTS ARE THE SAME
evolv = step(state.time[0])
print(math.abs(evolv-state.time[1]))

In [ ]:
# Path to training_input_ex4.csv
input_file_path = "./training_input_ex4.npy"
output_file_path = "./training_output_ex4.npy"
test_file_path =  "../input/advanced-dl-for-physics-assignment-4/test_input_ex4.npy"
# Loads training input, 100 training samples (batch), each sample is a 20-step time-series, axis is the spatial length (33 discrete point), and provides x&y projections
#loaded_input = numpy.genfromtxt(input_file_path, delimiter=',').reshape(100,20,33,2)
loaded_input = np.load(input_file_path).reshape(Nb,Nt,33,2)
observation = tensor(loaded_input, batch(batch=Nb,time=Nt), spatial(axis=33), channel(projection='x,y'))

# Loads training labels, 100 training samples (batch), each sample is a 20-step time-series, 33x33 domain, contains smoke density (rho), and velocity (vx, vy)
#loaded_output = numpy.genfromtxt(output_file_path, delimiter=',').reshape(100,20,33,33,3)
loaded_output = numpy.load(output_file_path).reshape(Nb,Nt,33,33,3)
label = tensor(loaded_output, batch(batch=Nb,time=Nt), spatial(x=33, y=33), channel(features='rho,vx,vy'))

print(observation)
print(label)

loaded_test_input = np.load(test_file_path).reshape(-1,20,33,2)
observation_test = tensor(loaded_test_input, batch(batch=loaded_test_input.shape[0],time=20), spatial(axis=33), channel(projection='x,y'))
print(observation_test)

In [ ]:
reconstruction = math.random_normal(observation.shape.batch, spatial(x=33, y=33), channel(features='rho,vx,vy'))
reconstruction = math.random_normal(observation_test.shape.batch, spatial(x=33, y=33), channel(features='rho,vx,vy'))
# must have shape (batch, time=20, x=33, y=33, features=rho,vx,vy)

In [ ]:
#print(label.features['rho'])
proj_rho_x = math.sum(label.features['rho'], dim='y')
proj_rho_y = math.sum(label.features['rho'], dim='x')
proj = proj_rho_x + proj_rho_y

#model = u_net(2, 3, levels=4, filters=[4, 8, 8, 4], in_spatial=2)
class conv_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.add_module(f'conv_1',nn.Conv2d(2, 4, 3, 1, 1) )
        self.add_module(f'conv_2', nn.Conv2d(4, 4, 3, 1, 1))
        self.add_module(f'conv_skip1', nn.Conv2d(2, 4, 1))

        self.add_module(f'conv_3', nn.Conv2d(4, 8, 3, 1, 1))
        self.add_module(f'conv_4', nn.Conv2d(8, 8, 3, 1, 1))
        self.add_module(f'conv_skip3', nn.Conv2d(4, 8, 1))

        self.add_module(f'conv_5', nn.Conv2d(8, 16, 3, 1, 1))
        self.add_module(f'conv_6', nn.Conv2d(16, 16, 3, 1, 1))
        self.add_module(f'conv_skip5', nn.Conv2d(8, 16, 1))
        
        self.add_module(f'conv_7', nn.Conv2d(16, 32, 3, 1, 1))
        self.add_module(f'conv_8', nn.Conv2d(32, 32, 3, 1, 1))
        self.add_module(f'conv_skip7', nn.Conv2d(16, 32, 1))
        
        self.add_module(f'conv_9', nn.Conv2d(32, 16, 3, 1, 1))
        self.add_module(f'conv_10', nn.Conv2d(16, 16, 3, 1, 1))
        self.add_module(f'conv_skip9', nn.Conv2d(32, 16, 1))
        
        self.add_module(f'conv_11', nn.Conv2d(16, 8, 3, 1, 1))
        self.add_module(f'conv_12', nn.Conv2d(8, 8, 3, 1, 1))
        self.add_module(f'conv_skip11', nn.Conv2d(16, 8, 1))
        
        self.add_module(f'conv_13', nn.Conv2d(8, 4, 3, 1, 1))
        self.add_module(f'conv_14', nn.Conv2d(4, 4, 3, 1, 1))
        self.add_module(f'conv_skip13', nn.Conv2d(8, 4, 1))

        self.add_module(f'conv_15', nn.Conv2d(4, 3, 3, 1, 1))
        self.add_module(f'conv_16', nn.Conv2d(3, 3, 3, 1, 1))
        self.add_module(f'conv_skip15', nn.Conv2d(4, 3, 1))


    def forward(self, x):
        for i in range(1,15, 2):
            x1 = F.relu(getattr(self, f'conv_{i}')(x)) 
            x = F.relu(getattr(self, f'conv_{i+1}')(x1) + getattr(self, f'conv_skip{i}')(x))
        x1 = F.relu(getattr(self, 'conv_15')(x))
        x = getattr(self, 'conv_16')(x1) + getattr(self, 'conv_skip15')(x) 
        return x 


model = conv_net().to('cuda')
#model = U_net(2, 3, 6)
print('Model Parameter Count:')
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
optim = adam(model, learning_rate=1e-3)
def loss_fn(proj, u):
    #proj = math.sum(u.features['rho'], dim='y') + math.sum(u.features['rho'], dim='x')
    loss = 0
    
    for i in range(10):
        #print(i)
        proj = math.expand(math.rename_dims(proj, 'axis', 'axis_x') , spatial(axis_y=33))
        pred = math.native_call(model, proj)
        #print('proj_shape')
        #print(proj.shape)
        #print(pred.shape)
        pred = math.tensor(pred, batch(batch=Nb), channel(features='rho,vx,vy'), spatial(x=33, y=33))
        #print(pred)
        loss += math.l2_loss(pred - u)
        #u = step(u)
        u = step(u)
        #print(u_m.shape)
        proj_x = math.sum(pred.features['rho'], dim='y')
        proj_x = math.reshaped_native(proj_x, ('batch', 'x'))
        proj_y = math.sum(pred.features['rho'], dim='x')
        proj_y = math.reshaped_native(proj_y, ('batch', 'y'))
        
        proj_x = torch.unsqueeze(proj_x, 2)
        proj_y = torch.unsqueeze(proj_y, 2)
        proj = torch.concat([proj_x, proj_y], 2)
        proj = math.tensor(proj,batch(batch=Nb), spatial(axis=33) , channel(projection='x,y'))
    return math.sum(loss, 'batch')

num_epochs = 10
for epoch in range(num_epochs):
    for t in range(0,Nt):
        proj = observation.time[t]
        u = label.time[t]
        #print('Training Loop Shapes:')
        #print(proj.shape)
        #print(u.shape)
        loss = update_weights(model, optim, loss_fn, proj, u)
        if t%5==0:
            print(f'Epoch: {epoch}, Time Step: {t},Loss: {loss}')

#reconstruction = math.native_call(model, observation_test)

In [ ]:

Nbt = 1 #Test Batches
proj = math.expand(math.rename_dims(observation_test.time[0], 'axis', 'axis_x') , spatial(axis_y=33))
#print(proj.shape)
pred = math.native_call(model, proj)

pred = math.tensor(pred, batch(batch=Nbt), channel(features='rho,vx,vy'), spatial(x=33, y=33))

reconstruction = pred
#print(pred.shape)
reconstruction = math.expand(reconstruction, batch(time=1))
for t in range(1,20):
    proj = math.expand(math.rename_dims(observation_test.time[t], 'axis', 'axis_x') , spatial(axis_y=33))
    pred = math.native_call(model, proj)
    
    pred = math.tensor(pred, batch(batch=Nbt), channel(features='rho,vx,vy'), spatial(x=33, y=33))
    pred = math.expand(pred, batch(time=1))
    
    reconstruction = math.concat( [reconstruction, pred], 'time')

print(reconstruction.shape)
reconstruction = math.reshaped_native(reconstruction, ('time', 'batch', 'x', 'y', 'features'))#batch(time=20, batch=1),spatial(x=33, y=33), channel(features='rho,vx,vy'))
reconstruction = torch.permute(reconstruction, [1, 0, 2,3,4])
reconstruction = math.tensor(reconstruction, batch(batch=1, time=20), spatial(x=33, y=33), channel(features='rho,vx,vy'))
print(reconstruction.shape)

**Do not edit the following cell!**
Finally, we compute the physics and projection match loss. The following cell must execute last in your notebook.

In [ ]:
assert reconstruction.shape == observation_test.shape.batch & spatial(x=33, y=33) & channel(features='rho,vx,vx'), "Reconstruction does not have the correct shape!"

from phi.field._grid import unstack_staggered_tensor
def cen_to_stag(centered_grid):
    padded_values = math.pad(centered_grid.values, {'x':(0,1), 'y':(0,1)}, mode=math.extrapolation.ZERO)
    return padded_values

def reduce_to_cen(reduce_tensor):
    return math.pad(reduce_tensor, {'x':(0,-1), 'y':(0,-1)}, mode=math.extrapolation.BOUNDARY)

def step(state, dt=5.):
    s = CenteredGrid(reduce_to_cen(state.features['rho']), extrapolation.BOUNDARY, x=32, y=32, bounds=Box(x=100, y=100))
    unstacked_velocity = unstack_staggered_tensor(math.rename_dims(state.features['vx,vy'], 'features', 'vector'), extrapolation=math.extrapolation.ZERO)
    v = StaggeredGrid(unstacked_velocity, 0, x=32, y=32, bounds=Box(x=100, y=100))
    s = advect.mac_cormack(s, v, dt) #+ INFLOW
    buoyancy = s * (0, 0.1) @ v  # resamples smoke to velocity sample points
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v,_= fluid.make_incompressible(v, (), Solve('auto', 1e-5, 0))
    v = v.staggered_tensor()
    return math.stack([cen_to_stag(s), v.vector['x'], v.vector['y']], channel(features='rho,vx,vy'))

phys_loss = math.l2_loss(step(reconstruction).time[:-1] - reconstruction.time[1:]).numpy('batch,time').flatten()
proj_x = math.sum(reconstruction.features['rho'], 'x').numpy('batch,time,y').flatten()
proj_y = math.sum(reconstruction.features['rho'], 'y').numpy('batch,time,x').flatten()
numpy.savetxt('result.csv', numpy.concatenate([phys_loss, proj_x, proj_y]), delimiter=',')




In [ ]:
result = np.concatenate([phys_loss, proj_x, proj_y], axis = 0)
print(result.shape)
submission_df = pd.DataFrame(data = {'Id':np.arange(result.shape[0]), 'Expected':result})
submission_df.to_csv('result.csv', index=False)